In [13]:
# USDC, WBTC, ETH
N_COINS = 3
PRECISION = 1e18

D = 29047349899039352915342907
balances = [9835488234445, 14938821663, 2811863139959144531319]
precisions = [1e12, 1e10, 1]
price_scale = [66266853956842562806222, 3465399375853261468310]

xp = [b for b in balances]

xp[0] *= precisions[0]
for k in range(1, N_COINS):
    xp[k] = xp[k] * price_scale[k - 1] * precisions[k] / PRECISION

for x in xp:
    print(x / 1e18)

9835488.234445
9899487.13429337
9744228.770199211


In [1]:
def get_xcp(D, prices):
    N = 3
    x = [0, 0, 0]
    x[0] = D / N

    for i in range(1, N):
        x[i] = D * 10**18 / (N * prices[i - 1])

    return (x[0] * x[1] * x[2]) ** (1/N)

get_xcp(D, price_scale) / 1e18

NameError: name 'D' is not defined

In [1]:
N = 3
ADMIN_FEE = 0.5

def newton_D(A, gamma, xp, K0_prev):
    return 1

def geo_mean(x):
    return (x[0] * x[1] * x[2]) ** (1/3)

class CurveV2:
    def __init__(self):
        self.xcp_profit = 1
        self.xcp_profit_a = 1
        self.price_scale = [1, 1, 1]
        self.A = 1
        self.gamma = 0.01
        self.total_supply = 0
        self.virtual_price = 1
        self.D = 0

    def xp():
        return [
            self.balances[0] * self.price_scale[0],
            self.balances[1] * self.price_scale[1],
            self.balances[2] * self.price_scale[2],
        ]

    def get_xcp(self, D):
        x = [
            D / (N * self.price_scale[0]),
            D / (N * self.price_scale[1]),
            D / (N * self.price_scale[2])
        ]
        return geo_mean(x)

    def get_virtual_price(self):
        return self.get_xcp(self.D) / self.total_supply

    def exchange(self):
        # swap
        xp = []
        y_out = [0, 0]
        self.tweak_price(self.A, self.gamma, xp, 0, y_out[1])

    def add_liquidity(amounts):
        A = self.A
        gamma = self.gamma
        xp = [
            (self.balances[0] + amounts[0]) * self.price_scale[0],
            (self.balances[1] + amounts[1]) * self.price_scale[1],
            (self.balances[2] + amounts[2]) * self.price_scale[2],
        ]
        D = newton_D(A, gamma, xp, 0)
        self.tweak_price(A, gamma, xp, D, 0)
        self._claim_admin_fees()

    def remove_liquiidty():
        self._claim_admin_fees()

    def tweak_price(self, A, gamma, xp, new_D, K0_prev):
        total_supply = self.total_supply
        old_xcp_profit = self.xcp_profit
        old_virtual_price = self.virtual_price
        price_scales = [
            self.price_scale[0],
            self.price_scale[1],
            self.price_scale[2],
        ]

        # Skip - update price_scale

        D_unadjusted = new_D
        if new_D == 0:
            D_unadjusted = newton_D(A, gamma, xp, K0_prev)

        xp = [0, 0, 0]
        for i in range(3):
            xp[i] = D_unadjusted / (N * price_scale[i])

        xcp_profit = 1
        virtual_price = 1

        if old_virtual_price > 0:
            xcp = geo_mean(xp)
            virtual_price = xcp / total_supply

            xcp_profit = old_xcp_profit * virtual_price / old_virtual_price
            assert virtual_price > old_virtual_price, "loss"

        self.xcp_profit = xcp_profit

        # Rebalance logic
        if virtual_price * 2 - 1 > xcp_profit:
            norm = 0
            adjustment_step = 0
            if norm > adjustment_step:
                # repeg and recalculate xp
                D = newton_D(A, gamma, xp, 0)
                
                new_virtual_price = geo_mean(xp) / total_supply
                if new_virtual_price > (1 + xcp_profit) / 2:
                    self.D = D
                    self.virtual_price = new_virtual_price
                    return

        self.D = D_unadjusted
        self.virtual_price = virtual_price


    def _claim_admin_fee(self):
        A = self.A
        gamma = self.gamma
        xcp_profit = self.xcp_profit
        xcp_profit_a = self.xcp_profit_a
        total_supply = self.total_supply

        if xcp_profit <= xcp_profit_a or total_supply < 1:
            return

        # Skip - sync self.balances with ERC20 balancess

        vprice = self.virtual_price
        fees = (xcp_profit - xcp_profit_a) * ADMIN_FEE / 2
        frac = vprice / (vprice - fees) - 1
        # Mint relative
        d_supply = total_supply * frac
        if d_supply > 0:
            self.total_supply += d_supply
        claimed = d_supply
        xcp_profit -= fees * 2
        self.xcp_profit = xcp_profit

        D = newton_D(A, gamma, self.xp(), 0)
        self.D = D
        self.virtual_price = self.get_xcp(D) / self.totalSupply
        self.xcp_profit_a = xcp_profit


In [2]:
amm = CurveV2()
amm.get_xcp(100)

33.33333333333333